# Flask Notes

## Installation and Environment Setup

- I have created a conda environment for running flask that contains most of my favorite/most used packages
- The steps below outline how to reproduce this environment from the Anaconda prompt

<p style="background:black;text-align:left">
    <code style="background:black;color:white;margin:3px"><br> (base) C:\Users\User\Documents> conda update -n base conda<br> (base) C:\Users\User\Documents> conda create -n flask_env python=3.8<br> (base) C:\Users\User\Documents> conda activate flask_env<br> (flask_env) C:\Users\User\Documents> conda install -c conda-forge notebook ipywidgets nodejs<br> (flask_env) C:\Users\User\Documents> conda install pandas scikit-learn geopandas<br> (flask_env) C:\Users\User\Documents> pip install plotly==4.9.0<br> (flask_env) C:\Users\User\Documents> pip install nbserverproxy paramiko pymongo JayDeBeApi xmltodict pgpy flask dash dask pytest coverage<br> (flask_env) C:\Users\User\Documents> conda install -c plotly plotly-orca psutil requests<br> (flask_env) C:\Users\User\Documents> jupyter serverextension enable --py nbserverproxy
    </code>
    </p>

## The Flask Class and Your app

- The Flask object implements a Web Server Gateway Interface (WSGI or WIZ-ghee) app and acts as the central object
- **Never name a flask app `flask.py`**
    - This will interfere with flask
- It is passed the name of the module or package of the application
    - Either the folder name with the `__init__.py` file inside or the name of a `.py` file
- Usually, you creat a Flask instance in the main module or in the `__init__.py` file like this:
> ```python
from flask import Flask
app = Flask(__name__)
```

#### More info on the parameter passed to `Flask()`
- If using a single module, *ALWAYS* pass `__name__`
- If you are using a package, it's usually best practice to hardcode the name of your package there
- Example: app is defined in `myapplication/app.py`
> ```python
app = Flask('myapplication') #or
app = Flask(__name__.split('.')[0])
```

## Flask Hello World App
```python
from flask import Flask

# create an instance of Flask
app = Flask(__name__)

# the route() decorator tells Flask what URL should trigger this function
@app.route('/')
def hello_world():
    return 'Hello World!'
```

## Running a Flask App
- This runs on a local host in the browser
- See [Deployment Options](https://flask.palletsprojects.com/en/1.1.x/deploying/#deployment)

- You can also use this syntax:
<p style="background:black;text-align:left">
    <code style="background:black;color:white;margin:3px"><br> C:\path\to\app> set FLASK_APP=filename.py<br> C:\path\to\app> python -m flask run
    </code>
    </p>

#### cmd or Anaconda Prompt
<p style="background:black;text-align:left">
    <code style="background:black;color:white;margin:3px"><br> C:\path\to\app> set FLASK_APP=filename.py<br> C:\path\to\app> flask run
    </code>
    </p>
    
#### Linux style terminal
<p style="background:black;text-align:left">
    <code style="background:black;color:white;margin:3px"><br> user@host:~/path/to/app$ export FLASK_APP=filename.py
 user@host:~/path/to/app$ flask run
    </code>
    </p>
    
#### PowerShell
<p style="background:black;text-align:left">
    <code style="background:black;color:white;margin:3px"><br> C:\path\to\app> $env:FLASK_APP=filename.py<br> C:\path\to\app> flask run
    </code>
    </p>
    
- Copy and paste the URL after the `Running on:` console output in your browser
- Use `ctrl + c` in the terminal window to stop the session

## Enabling Development/Debug Mode
- The app will update while being hosted with any code changes
- Debug mode will be enabled
> When setting the `FLASK_APP` variable, set `FLASK_ENV=development`
- You can set back to the default value `FLASK_ENV=production` if wanting to change this

## App Routing and URL's
- Specify meaningful routes to your app's urls
- Examples:

```python
@app.route('/')
def index():
    return 'Index Page'

@app.route('/hello')
def hello():
    return 'Hello World'
```

## URL Variables
- Can add variable sections to a URL using `<variable_name>`
- The function receives the `<variable_name>` as a keyword arg
- Can also use a converter to specify the type fo the argument with `<converter:variable_name>`
    - Converter types:
        - string (default)
        - int
        - float
        - path
        - uuid
> Note: Always use markupsafe in conjunction with this to avoid injection attacks (see example)
        
```python
from markupsafe import escape

# variable passed as keyword arg to function
@app.route('/user/<username>')
def show_user_profile(username):
    # show the user's profile
    return f'User {escape(username)}'

# when post_id is always an integer
@app.route('/post/<int:post_id>')
def show_post(post_id):
    # show the post with teh specified id
    return f'Post {post_id}'

@app.route('/path/<path:subpath>')
def show_subpath(subpath):
    # show the subpath after /path/
    return f'Subpath {escape(subpath)}'
```

## URL Redirection Behavior
- Trailing slashes after a path affect what happens
```python
# points to a directory (folder) like location
# if you don't use the /, it redirects using the slash
@app.route('/projects/')
def projects():
    return 'The Project page'
```

```python
# no trailing / is like pointing to a file
# if you access this page with a /, 404 Not Found error, must omit slash in url to access
@app.route('/about')
def about():
    return 'The about page'
```

## URL Building
- The `url_for()` function is used to build URL's to specific functions
- First arg is the function name, additional kwargs are variables for the URL
    - Unknown variable parts are appended as query parameters
- See example below that prints out urls using the url_for function

In [1]:
from flask import Flask, url_for
from markupsafe import escape

# initialize the app
app = Flask(__name__)

# definte the index route function
@app.route('/')
def index():
    return 'index'

# define the login route function
@app.route('/login')
def login():
    return 'login'

# define the user page function
@app.route('/user/<username>')
def profile(username):
    return f"{escape(username)}'s profile"

# use test_request_context() function to tell Flask to behave as if it's handling a request
with app.test_request_context():
    # 'index' is the name of the function for which to get the url, then print
    print(url_for('index'))
    
    print(url_for('login'))
    print(url_for('login', next='/'))
    print(url_for('profile', username='John Doe'))

/
/login
/login?next=%2F
/user/John%20Doe


- Results above explained
    - Index and login should be straightforward
    - Login with next arg, next is passed as a variable to the URL, and / was encoded as %2F
        - Since the 'next' item isn't an arg for the login function, it was passed in the url
    - Profile passed the username as part of the url (not a variable) since it was given as an arg for the profile function

## HTTP Methods
- By default, `route()` only answers to GET requests
    - Use the `methods` arg of `route()` to handle different methods
    
```python
from flask import request

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        return do_the_login()
    else:
        return show_the_login_form()
```

- Notes
    - If GET is present, Flask auto-adds support for the HEAD method and handles HEAD requests according to [HTTP RFC](https://www.ietf.org/rfc/rfc2068.txt)
    - OPTIONS is also auto-added

## Module App vs. Package App
- Module App Structure<br>
<pre>/application.py</pre>
<pre>/templates</pre>
<pre style={'tab-size': 4}>    /hello.html</pre>
<pre>/static</pre>
<pre style={'tab-size': 4}>    /style.css</pre>
<br>
- Package App Structure<br>
<pre>/application</pre>
<pre style={'tab-size': 4}>    /__init__.py</pre>
<pre style={'tab-size': 4}>    /templates</pre>
<pre style={'tab-size': 4}>        /hello.html</pre>
<pre style={'tab-size': 4}>    /static</pre>
<pre style={'tab-size': 4}>        /style.css</pre>

## Static Files
- Like .js, .css, and image files
- Put them in the `/static/` directory next to your module or within your package

```python
# generating urls for static files
# filename is static/style.css
url_for('static', filename='style.css')
```

## Rendering Templates
- Use instead of trying to generate html within python (requires escaping)
- Jinja is configured to *auto-escape* any user inputs rendered in HTML
- Use the `render_template` option
- Put them in the `templates` folder next to your module app or inside your package app
- Uses the full power of [Jinja2 Templates](http://jinja.pocoo.org/docs/templates/)

```python
from flask import render_template

@app.route('/hello/')
@app.route('/hello/<name>')
def hello(name=None):
    return render_template('hello.html', name=name)
```

#### Jinja Basic Syntax
- Anything between `{{` and `}}` is an expression that will be output
- `{%` and `%}` represent control flow (like if/for)
- Blocks are denoted with start/end tags rather than indentation

#### Jinja Blocks for Templates
- Format is `{% block ##### %}`
    - `title` -> the title displayed in the browser's tab and window
    - `header` -> the title displayed on the page
    - `content` -> where the content of the page goes

#### Example Template Using Jinja Syntax
```html
<!doctype html>
<title>Hello from Flask</title>
{% if name %}
  <h1>Hello {{ name }}!</h1>
{% else %}
  <h1>Hello World!</h1>
{% endif %}
```

#### Template Locations
- Base templates usually go in the templates dir
- Templates for a blueprint usually go in a dir within templates that has the same name as the blueprint

#### More Info on Templates
- You have access to the `requests`, `session`, and `g` objects as well as the `get_flashed_messages()` function
    - [requests](https://flask.palletsprojects.com/en/1.1.x/api/#flask.request)
    - [session](https://flask.palletsprojects.com/en/1.1.x/api/#flask.session)
    - [g](https://flask.palletsprojects.com/en/1.1.x/api/#flask.g)
    - [get_flashed_messages()](https://flask.palletsprojects.com/en/1.1.x/api/#flask.get_flashed_messages)
- Templates are especially useful if inheritance is used
    - [Template Inheritance](https://flask.palletsprojects.com/en/1.1.x/patterns/templateinheritance/#template-inheritance)
    - Can keep header, navigation, and footer on all pages

## Accessing Request Data
- Flask uses Context Locals
    - Makes life easier unless unit testing
        - Use `with app.test_request_context()` to bind test requests for unit testing

#### Example of accessing/using the request object
```python
from flask import request

@app.route('/login', methods=['POST', 'GET'])
def login():
    error = None
    if request.method == 'POST':
        # this is a function defined elsewhere and used here for example
        # this function accesses the form attribute of the request object and gets two items that were submitted
        if valid_login(request.form['username'], request.form['password']):
            return log_the_user_in(request.form['username'])
        else:
            error = 'Invalid username/password'
    
    # this is execute if the request is a GET method
    # or if the log_the_user_in function returns False
    return render_template('login.html', error=error)
```

#### Accessing keys from the form attribute
- `KeyError` is raised if the key doesn't exist in the form, resulting in HTTP 400 Bad Request page if not caught
```python
# best practice to use .get or catch KeyErrors
searchword = request.args.get('key', '')
```

## File Uploads
- Be sure to set `enctype='multipart/form-data'` attribute on your html form, or the browser won't transmit the files
- Uploaded files are stored in memory or a temp location on the filesystem
- Access them using the `request.files` attribute
- Files behave like a python file object
    - In addition has a `save()` method to store on the filesystem of the server

#### File Upload Examples
- More examples [here](https://flask.palletsprojects.com/en/1.1.x/patterns/fileuploads/#uploading-files)

```python
from flask import request

@app.route('/upload')
def upload_file():
    if request.method == 'POST':
        f = request.files['the_file']
        f.save('/var/www/uploads/uploaded_file.txt')
```

#### Filename Attribute
- Filenames can be forged, so never trust this attribute
- There is a `secure_filename()` function Werkzeug provides

```python
from flask import request
from werkzeug.utils import secure_filename

@app.route('/upload', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        f = request.files['the_file']
        f.save('/var/www/uploads/' + secure_filename(f.filename))
```

## Cookies
- Access with the `cookies` attribute
- Can `set_cookie`
- Different than `Sessions` in flask, so don't use if using `Sessions`
    - Sessions are usually better for managing user data

```python
# reading cookies
from flask import request

@app.route('/')
def index():
    username = request.cookies.get('username')
    # .get avoides keyerrors
```

```python
# setting cookies
from flask import make_response

@app.route('/')
def index():
    resp = make_response(render_template(...))
    resp.set_cookie('username', 'a user')
    return resp
```

## Redirects and Errors
- See [Error Handlers](https://flask.palletsprojects.com/en/1.1.x/errorhandling/#error-handlers) for more details

```python
from flask import abort, redirect, url_for

@app.route('/')
def index():
    
    # this function will redirect no matter what (pointless)
    # but can use this in if statements or other uses
    return redirect(url_for('login'))

# even more pointless since the redirect to login will abort to a page that the user can't access
@app.route('/login')
def login():
    
    # abort will display an error page based on the number provided, 401 is accessed denied
    # default is a black and white error page
    abort(401)
    this_is_never_executed()
```

```python
from flask import render_template

# use errorhandler() to customize the error page
@app.errorhandler(404)
def page_not_found(error):
    return render_template('page_not_found.html'), 404

# the 404 after render_template sets the status code of the page (404 == not found)
# default status code == 200 (all is well)
```

## View Functions
- Code to respond to requests to the application
- Flask uses patterns to match incoming request URL to the view that should handle it
- The view returns data that Flask turns into an outgoing response
- Flask can also generate a URL to view based on its name and args

## Blueprints
- A way to organize a group of related views and other code
- Register views/code with a blueprint instead of the app
- Register the blueprint with the app in the factory function

## About Response Objects
- The return value from a view function (`app.route()` I think) is automatically converted into a response object
- If the return value is a string, it's converted into a response object with the string as the response body, 200 status code, and text/html mimetype
- If the return value is a dict, jsonify() is called to produce a response
- Logic
    1. If a response object of the correct type is returned it's directly returned from the view
    2. If it's a string, a response object is created with that data and default parameters
    3. If it's a dict, a response object is created using `jsonify`
    4. If a tuple is returned, the items in the tuple can provide extra information
        - must be in one of the following forms
            - `(response, status)`
            - `(response, headers)`
            - `(response, status, headers)`
        - the `status` will override the status code and `headers` can be a list or dictionary of additional header values
    5. If nothing works, flask will assume the return value is a valid WSGI application and convert that into a response object

#### Work with the response object inside the view to modify it
- Use `make_response()`

```python
@app.errorhandler(404)
def not_found(error):
    resp = make_response(render_template('error.html'), 404)
    resp.headers['X-Something'] = 'A value'
    return resp
```

## APIs with JSON
- A common response when writing an API is JSON
- Return a dict to convert it to a JSON response

```python
@app.route('/me')
def me_api():
    user = get_current_user()
    return {
        'username': user.username,
        'theme': user.theme,
        'image': url_for('user_image', filename=user.image),
    }

# or for other purposes, use jsonify
@app.route('/users')
def users_api():
    users = get_all_users()
    return jsonify([user.to_json() for user in users])
```

## Sessions
- Better than cookies because users can look at cookie contents but not modify them unless they know the secret key used for signing
- A good way to generate random keys below

```python
import os

print(os.urandom(16))
```

```python
# session example

from flask import Flask, session, redirect, url_for, request
from markupsafe import escape

app = Flask(__name__)

# set the secret key to some random bytes (keep secred or encrypted using pgp)
app.secret_key = b'_dsk43#kUd/zKd//lkd]'

@app.route('/')
def index():
    if 'username' in session:
        return f'Logged in as {escape(session["username"])}'
    return 'You are not logged in'

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        session['username'] = request.form['username']
        return redirect(url_for('index'))
    return '''
        <form method='post'>
            <p><input type=text name=username></p>
            <p><input type=submit value=Login></p>
        </form>
        '''

@app.route('/logout')
def logout():
    # remove the username from the session if it's there
    session.pop('username', None)
    return redirect(url_for('index'))
```

A note on cookie-based sessions: Flask will take the values you put into the session object and serialize them into a cookie. If you are finding some values do not persist across requests, cookies are indeed enabled, and you are not getting a clear error message, check the size of the cookie in your page responses compared to the size supported by web browsers.

Besides the default client-side based sessions, if you want to handle sessions on the server-side instead, there are several Flask extensions that support this.

## Flashing Messages
- You can 'record a message' at the end of a request and access it on the next and only next request
- Note: flashing messages too large for session cookies causes message flashing to fail
- More details on flashing [here](https://flask.palletsprojects.com/en/1.1.x/patterns/flashing/#message-flashing-pattern)

#### Flashing Example for a Login

```python
from flask import Flask, flash, redirect, render_template, request, url_for

app = Flask(__name__)
app.secret_key = b'_48dkKDN/dikx]/'

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    error = None
    if request.method == 'POST':
        if request.form['username'] != 'admin' or request.form['password'] != 'secret':
            error = 'Invalid Credentials'
        else:
            
            # flashing message
            flash('You were successfully logged in')
            return redirect(url_for('index'))
    return render_template('login.html', error=error)
```

```html
<!-- the layout.html template which does the magic -->
<!doctype html>
<title>My Application</title>
{% with messages = get_flashed_messages() %}
  {% if messages %}
    <ul class=flashes>
        {% for message in messages %}
        <li>{{ message }}</li>
        {% endfor %}
    </ul>
  {% endif %}
{% endwith %}
{% block body %}{% endblock %}
```

```html
<!-- the index.html template which inherits the layout.html form -->
{% extends 'layout.html' %}
{% block body %}
  <h1>Overview</h1>
  <p>Do you want to <a href="{{ url_for('login') }}">log in?</a></p>
{% endblock %}
```

```html
<!-- login.html file which inherits layout.html -->
{% extends 'layout.html' %}
{% block body %}
  <h1>Login</h1>
  {% if error %}
    <p class=error><strong>Error:</strong>  {{ error }}</p>
  {% endif %}
  <form method=post>
      <dl>
          <dt>Username:
          <dd><input type=text name=username value="{{ request.form.username }}">
          <dt>Password:
          <dd><input type=password name=password>
      </dl>
      <p><input type=submit value=Login></p>
  </form>
{% endblock %}
```

## Flask Logging
- Flask has a built-in logger that looks similar to the logger I've used before
- See [Flask Logger](https://flask.palletsprojects.com/en/1.1.x/api/#flask.Flask.logger)
- See also [Application Errors](https://flask.palletsprojects.com/en/1.1.x/errorhandling/#application-errors)

## Flask Extensions
- [Flask Extensions](https://flask.palletsprojects.com/en/1.1.x/extensions/#extensions)

## More Details on Flask Workings
- See the app in /flask-tutorial
    - flaskr/ (package)
        - static/
            - style.css
        - templates/
            - auth/
                - login.html
                - register.html
            - blog/
                - create.html
                - index.html
                - update.html
            - base.html
                - the base template that is extended by other templates in auth/ and blog/
        - __init__.py (package identifier containing the app factory function)
        - auth.py
            - authentication functions blueprint
            - registered in factory function
        - blog.py
            - blog blueprint 
            - registered in factory function
        - db.py
            - includes a click.command() that defines a command line command [Command Line Interface](https://flask.palletsprojects.com/en/1.1.x/cli/#cli)
            - init_db function registered with factory function
        - schema.sql
            - DDL used to create/restore user database with SQLite
    - instance/ (instance folder)
        - flaskr.sqlite
            - created when `flask init-db` command was run from the cli in flask-tutorial directory along with `instance` folder
            - that command executed the `db.init_db()` function which ran the sql in `/flaskr/schema.sql`
    - tests/
        - conftest.py
            - code needed to configure the unit tests when running in test mode
            - also created a temp database file to hold dummy data for the tests
        - data.sql
            - code to add dummy data to the temp_db for the unit tests
        - test_auth.py
            - tests for the authorization
        - test_blog.py
            - tests for the blog
        - test_db.py
            - tests for the database
        - test_factory.py
            - test functions for the app factory
    - venv/
        - contains the virtual environment
    - MANIFEST.in
        - contains data needed to run <code style="background:black;color:white;margin:3px">pip install -e .</code> to install the app (see below)
    - README.md
        - contains info about the app
    - setup.cfg
        - unrequired, but sets up some unit test testing parameters
    - setup.py
        - contains info necessary to run <code style="background:black;color:white;margin:3px">pip install -e .</code> to install the app (see below)

## Installing Custom Flask Apps
- Requires the `setup.py` and `MANIFEST.in` files as outlined in the `flask-tutorial` app
- Run the following command from the `flask-tutorial` dir with a virtual env activated:
<br><code style="background:black;color:white;margin:3px">pip install -e .</code>
- Installing will allow the app to run from any dir using `flask run` after set/export the `FLASK_APP=flaskr` and `FLASK_ENV` vars
    - You also need to intialize any app specific files like running `flask init-db` for the flask-tutorial app
- More info on packaging python projects can be found [here](https://packaging.python.org/tutorials/packaging-projects/)
- This is required for running unit tests built the same way the flaskr unit test is implemented

## Notes on Unit Testing
- Ensure `pytest` and `coverage` are installed in the venv
- Ensure you have installed the flask app in the venv using the instructions above
- Running `pytest` in the project dir will run the unit tests
- Running `pytest -v` will show each function name and the result
- Running `coverage run -m pytest` will measure the code coverage
- Run `coverage report` to get a simple output of this
- Run `coverage html` if you want to generate html files in the `htmlcov` dir within the project
    - view the `index.html` file for the report

## Deploying to a Production Server

### Build and Install
- Ensure wheel is installed `pip install wheel`
- From the main app dir run `python setup.py bdist_wheel`
    - this should create `dist/flaskr-1.0.0-py3-none-any.whl` for the flask-tutorial app
- Now you can install on another machine using these steps
    - create and activate a new virtual env on the new machine
    - run `pip install flaskr-1.0.0-py3-none-any.whl`
    - run either `set/export FLASK_APP=flaskr` and `flask init-db`
    - Note: the `instance` dir is located at `venv/var/flaskr-instance` instead of in the main app dir
    - Configure the secret key
        - create `venv/var/flaskr-instance/config.py` and add `SECRET_KEY = b''` using random bytes chars
            - you can generate these using `python -c 'import os; print(os.urandom(16))'`
        - configure any other vars that are needed here too for custom apps
- Use a production server to serve the app
    - See [Deployment Options](https://flask.palletsprojects.com/en/1.1.x/deploying/) for some examples

## Further Flask Exploration
- [Jinja](https://palletsprojects.com/p/jinja/)
- [Click](https://palletsprojects.com/p/click/)
- [Werkzeug](https://palletsprojects.com/p/werkzeug/)
- [ItsDangerous](https://palletsprojects.com/p/itsdangerous/)
- [Extensions](https://flask.palletsprojects.com/en/1.1.x/extensions/#extensions)